## <font color=green> Praticando: Analisando o Dataset do Titanic

In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
%config Completer.use_jedi = False

### Carregando Dataset

In [3]:
df_titanic = pd.read_csv('titanic.csv')
df_titanic.shape

FileNotFoundError: [Errno 2] No such file or directory: 'titanic.csv'

In [ ]:
df_titanic.head()

- **Passengerid**: ID do passageiro do navio (código primário).
- **Survived**: Se sobreviveu ao naufrágio estará como 1 e caso esteja com 0 (zero) não sobreviveu.
- **Pclass**: Tipo de classe de passagem (Do 1 ao 3), sendo 1 a melhor classe e 3 a pior classe.
- **Name**: Nome do passageiro
- **Sex**: Gênero do passageiro, sendo masculino e feminino.
- **Age**: Idade do passageiro na data da ocorrência do naufrágio.
- **SibSp**: Número de irmãos / cônjuges a bordo.
- **Parch**: Número de pais / filhos a bordo.
- **Ticket**: Código do ticket.
- **Fare**: Valor da passagem.
- **Cabin**: Código de identificação da Cabine.
- **Embarked**: Local onde o passageiro embarcou no navio.

### Verificando os tipos de dados contidos no Dataset

In [ ]:
df_titanic.dtypes

In [ ]:
df_titanic.describe().T

- A variável idade tem como valor máximo 80. Contudo o 3º quartil mostra a idade de 39 anos, o que indica que há outliers
- Quanto a coluna Fare (valor da passagem), temos outliers distorcendo a distribuição dos valores desta variável, puxando a média para 4428.39 sendo que a mediana está em 20.08

### Explorando os dados categóricos e numéricos

In [ ]:
cat_col = [var for var in df_titanic.columns if df_titanic[var].dtype == 'O']
df_titanic.loc[:,cat_col].head()

In [ ]:
num_col = [var for var in df_titanic.columns if df_titanic[var].dtype != 'O']
df_titanic.loc[:,num_col].head()

- 3 variáveis discretas: Pclass, SibSp e Parch
- 2 variáveis contínuas: Fare e Age
- 1 variável Id: PassengerId (chave primária).
- 1 binário: Survived (booleano).

- Percentual de dados faltantes na base de treino

In [ ]:
round(df_titanic.isnull().mean()*100, 2)

- Podemos observar que há 3 colunas com dados faltantes: Sex, Age e Cabin

## Vamos explorar o dataset para analisar o que é possível fazer para preencher os dados faltantes

- Total de Passageiros

In [ ]:
df_titanic.shape[0]

- Total de Passageiros com todos os registros preenchidos (linhas)

In [ ]:
df_titanic.dropna().shape[0]

- Percentual de dados com 100% do preenchimento dos dados

In [ ]:
round(df_titanic.dropna().shape[0] / len(df_titanic)*100, 2)

- Selecionando os registros com dados ausentes para a coluna Pclass

In [ ]:
df_titanic[df_titanic['Pclass'].isnull()].shape[0]

- Não temos nenhum registro faltante para a coluna Pclass

- É interessante conseguir extrair os títulos dos nomes para verificar se temos realmente correlação entre as variáveis

In [ ]:
def extrai_titulo(df):
    df['Título'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip().lower())

extrai_titulo(df_titanic)

- Checando se a coluna Título foi criada corretamente

In [ ]:
df_titanic.head()

- Checando se há algum registro sem título

In [ ]:
df_titanic[df_titanic['Título'].isnull()].shape[0]

In [ ]:
fig = df_titanic['Título'].value_counts().plot.barh(figsize=(11,6))
fig.set_title('Abreviações dos Títulos dos Passageiros')
fig.set_ylabel('Nome dos Títulos')
fig.set_xlabel('Quantidade por Título')

df_titanic['Título'].value_counts()

- Conforme o gráfico acima, temos vários títulos

- Vamos investigar os outliers como já identificamos acima nas variáveis Age e Fare. Para isso, vamos utilizar o boxplot.

In [ ]:
df_titanic['Age'].plot.box(vert=False, figsize=(15,2))

In [ ]:
df_titanic['Fare'].plot.box(vert=False, figsize=(15,2))

- Como já esperado, diversos Outliers

- Para a coluna Fare, a quantidade de outiliers é tão grande que não conseguimos visualizar o box. Portanto, vamos filtrar por Fare menor que 500.

In [ ]:
df_titanic['Fare'].loc[df_titanic['Fare'] < 500].plot.box(vert=False, figsize=(15,2))

- Analisando a distribuição das variáveis para descobrir se elas são gaussianas (normais) ou distorcidas (assimétricas)

In [ ]:
df_titanic['Age'].hist(bins = 25, figsize=(12,6))

In [ ]:
df_titanic['Fare'].loc[df_titanic['Fare'] < 500].hist(bins = 25, figsize=(12,6))

- Vamos ver a distribuição de pessoas por tipo de classe de passagem

In [ ]:
df_titanic['Pclass'].value_counts().plot.barh(figsize=(11,6))

- Vamos identificar qual o nosso *threshold* de valor da passagem pago apenas pelos passageiros da Primeira Classe

In [ ]:
df_titanic.loc[df_titanic['Fare'] > 10000].groupby(['Pclass','Fare'])[['PassengerId']].count()

- Podemos perceber que o valor mais alto pago na terceira classe foi de 34375. Ou seja, todo mundo que pagou um valor acima desse está na primeira classe.

- Vamos identificar agora a distribuição das classes das pessoas pelos seus títulos

In [ ]:
df_titanic.groupby(['Título','Pclass']).count()

- Todos que estão na terceira classe possuem o título de ms, mrs, mr e master. Todos os outros estão na segunda ou primeira classe.

In [ ]:
df_titanic.loc[df_titanic['Fare'] > 50000]['Pclass'].value_counts()

- Vamos encontrar a média de preço para os passageiros da primeira, segunda e terceira classe (exlcuindo os Outliers para não subir a média)

- 1ª classe

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 500) & (df_titanic['Pclass'] == 1)].plot.box(vert=False, figsize=(15,2))

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 200) & (df_titanic['Pclass'] == 1)].mean()

- 2ª classe

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 500) & (df_titanic['Pclass'] == 2)].plot.box(vert=False, figsize=(15,2))

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 60) & (df_titanic['Pclass'] == 2)].mean()

- 3ª classe

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 500) & (df_titanic['Pclass'] == 3)].plot.box(vert=False, figsize=(15,2))

In [ ]:
df_titanic['Fare'].loc[(df_titanic['Fare'] < 30) & (df_titanic['Pclass'] == 3)].mean()

### Tratando dados ausentes da coluna Sex

- Como poderíamos fazer isso?

- Uma alternativa é identicar seu gênero de acordo com o título do passageiro

In [ ]:
df_titanic['Título'].unique()

- Verificando se há algum passageiro que não possui título

In [ ]:
df_titanic[df_titanic['Título'].isnull()].shape[0]

- Listando títulos masculinos

In [ ]:
list_male = ['mr', 'master', 'don', 'rev', 'dr', 'major', 'sir', 'col', 'capt', 'jonkheer']

- Filtrando resultado que não possuem definição na coluna Sex

In [ ]:
df_aux = df_titanic[df_titanic['Sex'].isnull()]
df_aux

- Atribuindo o gênero do passageiro de acordo com o seu título

In [ ]:
df_aux['Sex'] = df_aux['Título'].apply(lambda x: 'male' if x in list_male else 'female')
df_aux

- Combinando os dois dataframes

In [ ]:
df_titanic = df_titanic.combine_first(df_aux)

In [ ]:
df_titanic[df_titanic['Sex'].isnull()].shape[0]

- Podemos observar que todos os passageiros possui o registro de gênero

### Tratando a coluna Age (Idade) para valores faltantes

In [ ]:
df_titanic[df_titanic["Age"].isnull()].shape[0]

- Temos 263 passageiros sem idade

- Podemos inserir as faixas de idades conforme classe e gênero

In [ ]:
df_titanic.groupby(['Pclass','Sex'])[['Age']].mean()

- Ou podemos tratar a idade por classe e título. Irá dar mais trabalho, mas ficará mais próximo da realidade.

In [ ]:
df_aux = df_titanic.groupby(['Pclass','Título'], as_index=False)[['Age']].mean()
df_aux

- Como pode ser visto, apena ms da terceira classe não possui idade. Vamos assumir então para essa situação o valor de ms da segunda classe.

In [ ]:
df_aux['Age'].loc[27] = df_aux['Age'].loc[21]
df_aux

In [ ]:
df_titanic[df_titanic['Age'].isnull()].shape[0]

- Podemos ver que há 263 registros sem Age

- Vamos utilizar uma função que retorna a média de idade baseada na média de idade de classe e título.

In [ ]:
def impute_age(age_pclass_titulo):
    age = age_pclass_titulo[0]
    pclass = age_pclass_titulo[1]
    titulo = age_pclass_titulo[2]
    if pd.isnull(age):
        return df_aux['Age'].loc[(df_aux['Pclass']==pclass) & (df_aux['Título']==titulo)].values[0]
    else:
        return age

In [ ]:
df_titanic['Age'] = df_titanic[['Age','Pclass','Título']].apply(impute_age, axis = 1)

In [ ]:
df_titanic[df_titanic['Age'].isnull()].shape[0]

### Tratando a coluna Cabin

- Agora só temos a coluna Cabin com dados faltantes que possui ~33% dos dados

In [ ]:
round(df_titanic.isnull().mean()*100, 2)

In [ ]:
df_titanic.head(10)

- Vamos verificar como estão distribuídas as cabines por classe, selecionando apenas a letra inicial de cada observação

In [ ]:
df_titanic['Ale_cabin'] = df_titanic['Cabin'].str[0].str.lower().str.strip()

In [ ]:
df_titanic.groupby('Pclass')['Ale_cabin'].value_counts()

- Temos a seguinte distribuição das cabines por classe:
    - Classe 1 = a, b, c, d, e, t
    - Classe 2 = d, e, f
    - Classe 3 = e, g, f

- Podemos remover a coluna Ale_cabin pois ja extraimos o que precisamos

In [ ]:
del df_titanic['Ale_cabin']

- Vamos criar um dicionário onde a chave é a classe e os valores são as possíveis letras das cabines

In [ ]:
dict_classe = {1:['a', 'b', 'c', 'd', 'e', 't'],
               2:['d', 'e', 'f'],
               3:['e', 'g', 'h']}

- Filtrando registros de Cabin nula

In [ ]:
df_aux = df_titanic[df_titanic['Cabin'].isnull()]
df_aux

- Vamos imputar os dados com a função choice do numpy

In [ ]:
df_aux['Cabin'] = df_titanic['Pclass'].apply(lambda x: np.random.choice(dict_classe[x], size=1)[0])

In [ ]:
df_aux.head(5)

- Combinando os dois dataframes

In [ ]:
df_titanic = df_titanic.combine_first(df_aux)

In [ ]:
df_titanic[df_titanic['Cabin'].isnull()].shape[0]

- Podemos observar que todos os registros de Cabin possuem pelo menos um letra associada

In [ ]:
round(df_titanic.isnull().mean()*100, 2)

- E com isso, finalizamos nossa atividade de limpeza dos dados. Todas as colunas possuem dados.

- Vamos explorar visualmente nossos dados. Vamos visualizar as pessoas que sobreviveram baseado na classe e gênero

- Pessoas que sobreviveram

In [ ]:
survived = df_titanic.loc[df_titanic['Survived'] == 1]
percentual = pd.crosstab(survived.Sex,
                         survived.Pclass,
                         aggfunc = 'count',
                         values = survived.Survived)

percentual.T.plot.bar(rot=0)

- Pessoas que não sobreviveram

In [ ]:
not_survived = df_titanic.loc[df_titanic['Survived'] == 0]
percentual = pd.crosstab(not_survived.Sex,
                         not_survived.Pclass,
                         aggfunc = 'count',
                         values = not_survived.Survived)

percentual.T.plot.bar(rot=0)

- Conforme nosso gráfico acima, em todas as classes o percentual de mulheres sobrevivente é superior aos homens e fica mais expressivo quando olhamos para a primeria e segunda classe.


- Média de idade dos que sobreviveram e dos que morreram

In [ ]:
df_titanic.groupby('Survived')[['Age']].mean()